## Format CSV File to Retsediv Format

### Take 52 data subcarriers from 64 subcarriers

In [4]:
import pandas as pd
from tabulate import tabulate

filename = 'aavash1'
filepath = './Data/Wireless Eye Data/' + f'{filename}' + '.csv'
frame = pd.read_csv(filepath, sep=';', usecols=['amplitude', 'subcarrier'])
# print(tabulate(frame, tablefmt='simple'))

# Load the CSV file
df = pd.read_csv(filepath, sep=';', usecols=['timestamp', 'amplitude', 'phase', 'subcarrier'])

df['timestamp'] = df['timestamp'].astype(str)

# Filter for data subcarriers (1-26 and 38-63)
data_subcarriers = list(range(1, 27)) + list(range(38, 64))

# Filter the dataframe to keep only data subcarriers
df = df[df['subcarrier'].isin(data_subcarriers)]

# Create pivot tables for amplitude and phase
subcarrier_pivot = df.pivot(index='timestamp', columns='subcarrier', values='subcarrier')
amplitude_pivot = df.pivot(index='timestamp', columns='subcarrier', values='amplitude')
phase_pivot = df.pivot(index='timestamp', columns='subcarrier', values='phase')

# Reset the index to make timestamp a column
subcarrier_pivot.reset_index(inplace=True)
amplitude_pivot.reset_index(inplace=True)
phase_pivot.reset_index(inplace=True)

# Rename the columns appropriately
subcarrier_pivot.columns = ['timestamp'] + [f'subcarrier_{i}' for i in subcarrier_pivot.columns[1:]]
amplitude_pivot.columns = ['timestamp'] + [f'amplitude_{i}' for i in amplitude_pivot.columns[1:]]
phase_pivot.columns = ['timestamp'] + [f'phase_{i}' for i in phase_pivot.columns[1:]]

# Drop the timestamp column from the amplitude and phase pivot tables before concatenation
subcarrier_pivot.drop(columns=['timestamp'], inplace=True)
amplitude_pivot.drop(columns=['timestamp'], inplace=True)
phase_pivot.drop(columns=['timestamp'], inplace=True)

# Concatenate the two pivot tables horizontally
combined_df = pd.concat([subcarrier_pivot, amplitude_pivot, phase_pivot], axis=1)
# Remove column names
combined_df.columns = range(combined_df.shape[1])
# Save the resulting dataframe to a CSV file
save_filepath = './Data/Processed Data/' + f'{filename}' + '_processed.csv'
combined_df.to_csv(save_filepath, index=False, header=False)